# Repeat MEME Motif analysis for NR29 samples

In [ ]:
import os
import sys
import re
import pandas as pd
from Bio import SeqIO

print(sys.version)

# Plot distribution of intergenic vs coding peaks

In [ ]:
seq_length = 500
INFILE = "/Users/yunfei/2023_ChipSeq/NRA29/RNAseq_Chipseq_sorted/BfmR-ChIP-29_seed1.master_table.sorted.tsv"
df_peak = pd.read_csv(INFILE, sep='\t')
df_peak = df_peak[['chrom', 'summit_pos', 'general call']]
df_peak['seq_start'] = df_peak['summit_pos'] - (seq_length // 2)
df_peak['seq_end'] = df_peak['summit_pos'] + (seq_length // 2)

df_peak_activated = df_peak_all[df_peak['general call'] == 'activated']
df_peak_activated = df_peak_activated[['chrom', 'seq_start', 'seq_end']]
df_peak_repressed = df_peak_all[df_peak['general call'] == 'repressed']
df_peak_repressed = df_peak_repressed[['chrom', 'seq_start', 'seq_end']]

OUTDIR = "/Users/yunfei/2023_ChipSeq/MEME_analysis_NRA29_seed1/bed_file_for_MEME_500"
df_peak_activated.to_csv(os.path.join(OUTDIR, "BfmR-ChIP-49_seed1_activated_peak.bed"), sep='\t', header=False, index=False, mode='w')
df_peak_repressed.to_csv(os.path.join(OUTDIR, "BfmR-ChIP-49_seed1_repressed_peak.bed"), sep='\t', header=False, index=False, mode='w')

## Generate fasta file from bed files

In [ ]:
INDIR = "/Users/yunfei/2023_ChipSeq/MEME_analysis_NRA29_seed1/bed_file_for_MEME_500"
FASTA = "/Users/yunfei/2023_ChipSeq/references/Ab_all.fasta"
for file in os.listdir(INDIR):
    if re.match('.+bed$', file):
        infile = os.path.join(INDIR, file)
        outfile = os.path.join(INDIR, file.split('.')[0]+'.fa')
        cmd = " ".join(['bedtools getfasta', '-fi', FASTA, '-bed', infile, '>', outfile])
        print("FASTA output: " + outfile)
        os.system(cmd)

## Run MEME

In [ ]:
INDIR = "/Users/yunfei/2023_ChipSeq/MEME_analysis_NRA29_seed1/bed_file_for_MEME_500"
OUTDIR = "/Users/yunfei/2023_ChipSeq/MEME_analysis_NRA29_seed1/MEME-Chip_output"
DB = "/Users/yunfei/meme/motif_databases/PROKARYOTE/collectf.meme"
LOG = os.path.join(OUTDIR, 'meme-chip_log.txt')

os.makedirs(os.path.join(OUTDIR, 'fimo_output'), exist_ok=True)
os.makedirs(os.path.join(OUTDIR, 'html_output'), exist_ok=True)
for file in os.listdir(INDIR):
    if re.match('.+fa$', file):
        infile = os.path.join(INDIR, file)
        prefix = os.path.join(OUTDIR, file.split('.')[0])
        cmd = ' '.join(['meme-chip -meme-p 4', '-o', prefix, '-db', DB, infile, ">>", LOG, "2>&1"])
        print(cmd)
        os.system(cmd)